In [1]:
from fake_useragent import UserAgent
import requests
import pandas as pd
import datetime
import bs4
import time
import tqdm

## Обращение к сайту

In [26]:
link = 'https://zadolba.li/20210928'
fake_user_agent = None

In [27]:
response = requests.get(link, headers={'User-Agent': UserAgent().chrome})
html = response.content

In [29]:
# Создадим соответствующую функцию
def get_html(link, fake_user_agent=None):
    
    response = requests.get(link, headers={'User-Agent': fake_user_agent})
    if response.ok:
        return response.content
    else:
        return None

In [33]:
html = get_html(link, fake_user_agent)

## Сбор блоков историй

In [34]:
soup = bs4.BeautifulSoup(html, 'html.parser')

In [39]:
content_block = soup.find('div', {'class': 'content'})

In [41]:
story_blocks = content_block.find_all('div', {'class': 'story'})

In [42]:
len(story_blocks)

10

In [44]:
# Создадим соответствующую функцию
def get_story_blocks(html):
    
    soup = bs4.BeautifulSoup(html, 'html.parser')
    content_block = soup.find('div', {'class': 'content'})
    story_blocks = content_block.find_all('div', {'class': 'story'})
    return story_blocks

In [45]:
story_blocks = get_story_block(html)

## Получение характеристик статей

* название статьи,
* текст статьи,
* категорию статьи,
* дату выхода статьи в формате даты

In [46]:
story_block = story_blocks[0]

### Название статьи

In [56]:
story_block.h2.text

'Все\xa0силы отданы борьбе'

In [57]:
story_block.h2.text.replace('\xa0', ' ')

'Все силы отданы борьбе'

### Текст статьи

In [59]:
story_block.find('div', {'class': 'text'}).text

'\nА\xa0у\xa0меня задолбашка примерно такая\xa0же, как у\xa0недавней мотоциклистки. Только я\xa0— пешеход. Но\xa0тоже задолбали любители обогнать и\xa0потом уныло плестись. Кстати, не\xa0только автомобилисты.\nНо\xa0начнём все\xa0же с\xa0автомобилистов. Классическая ситуация: нерегулируемый ПП, я\xa0собираюсь переходить дорогу, не\xa0бегу, не\xa0выпрыгиваю из\xa0кустов, спокойно стою и\xa0жду. Вот авто, казалось\xa0бы, может затормозить и\xa0пропустить меня\xa0— но\xa0нет, он\xa0лучше газанет со\xa0всей дури, чтобы… проехать 100−200м и\xa0встать в\xa0пробке. Ну\xa0и\xa0что ты\xa0выиграл, дядя? Да\xa0ничего.\nА\xa0вот уже не\xa0автомобилисты. Тоже классика\xa0— стою в\xa0вагоне в\xa0метро, готовлюсь выходить. Двери ещё не\xa0начали открываться, но\xa0сзади меня уже пихают и\xa0толкают так, что я\xa0чуть было не\xa0падаю. После выхода из\xa0вагона те, кто только что меня толкал, неспешным лёгким шагом медленно фланируют по\xa0перрону в\xa0направлении выхода. Чудаки, вы\xa0зачем толкались

In [64]:
story_block.find('div', {'class': 'text'}).text.replace('\xa0', ' ').replace('\n', ' ').strip()

'А у меня задолбашка примерно такая же, как у недавней мотоциклистки. Только я — пешеход. Но тоже задолбали любители обогнать и потом уныло плестись. Кстати, не только автомобилисты. Но начнём все же с автомобилистов. Классическая ситуация: нерегулируемый ПП, я собираюсь переходить дорогу, не бегу, не выпрыгиваю из кустов, спокойно стою и жду. Вот авто, казалось бы, может затормозить и пропустить меня — но нет, он лучше газанет со всей дури, чтобы… проехать 100−200м и встать в пробке. Ну и что ты выиграл, дядя? Да ничего. А вот уже не автомобилисты. Тоже классика — стою в вагоне в метро, готовлюсь выходить. Двери ещё не начали открываться, но сзади меня уже пихают и толкают так, что я чуть было не падаю. После выхода из вагона те, кто только что меня толкал, неспешным лёгким шагом медленно фланируют по перрону в направлении выхода. Чудаки, вы зачем толкались-то? Что вы хотели этим выиграть? Тут вообще ни секунды выигрыша Или вот сегодня. На выходе из метро женщина идёт, расталкивая все

### Категория статьи

In [69]:
html_tags = story_block.find('div', {'class': 'tags'}).ul.find_all('li')
html_tags

[<li><a href="/tag/transport" itemprop="about">транспорт</a></li>,
 <li><a href="/tag/household" itemprop="about">быт</a></li>]

In [88]:
tags = [html_tag.a.text for html_tag in html_tags]

In [89]:
', '.join(tags)

'транспорт, быт'

### Дата выхода статьи

In [78]:
meta_block = story_block.find('div', {'class': 'meta'})
meta_block

<div class="meta">
<time class="date-time" datetime="2021-09-28T14:48:10+03:00" itemprop="datePublished">Сегодня, 14:48</time>
<div class="tags">
<i class="icon-tags"></i>
<ul>
<li><a href="/tag/transport" itemprop="about">транспорт</a></li>
<li><a href="/tag/household" itemprop="about">быт</a></li>
</ul>
</div>
</div>

In [82]:
datetime = meta_block.find('time').get('datetime')
datetime

'2021-09-28T14:48:10+03:00'

In [83]:
datetime.split('T')[0]

'2021-09-28'

In [95]:
# Создадим соответствующую функцию
def get_description(story_block):
    
    try:
        title = story_block.h2.text.replace('\xa0', ' ')
    except AttributeError:
        title = None
    
    try:
        text = story_block.find('div', {'class': 'text'}).text.replace('\xa0', ' ').replace('\n', ' ').strip()
    except AttributeError:
        text = None
        
    try:
        html_tags = story_block.find('div', {'class': 'tags'}).ul.find_all('li')
        tags = [html_tag.a.text for html_tag in html_tags]
        tags = ', '.join(tags)
    except AttributeError:
        tags = None
    
    try:
        meta_block = story_block.find('div', {'class': 'meta'})
        datetime = meta_block.find('time').get('datetime')
        date = datetime.split('T')[0]
    except AttributeError:
        date = None
    
    return {'title': title, 'text': text, 'tags': tags, 'date': date}

In [98]:
description = get_description(story_block)
description

{'title': 'Все силы отданы борьбе',
 'text': 'А у меня задолбашка примерно такая же, как у недавней мотоциклистки. Только я — пешеход. Но тоже задолбали любители обогнать и потом уныло плестись. Кстати, не только автомобилисты. Но начнём все же с автомобилистов. Классическая ситуация: нерегулируемый ПП, я собираюсь переходить дорогу, не бегу, не выпрыгиваю из кустов, спокойно стою и жду. Вот авто, казалось бы, может затормозить и пропустить меня — но нет, он лучше газанет со всей дури, чтобы… проехать 100−200м и встать в пробке. Ну и что ты выиграл, дядя? Да ничего. А вот уже не автомобилисты. Тоже классика — стою в вагоне в метро, готовлюсь выходить. Двери ещё не начали открываться, но сзади меня уже пихают и толкают так, что я чуть было не падаю. После выхода из вагона те, кто только что меня толкал, неспешным лёгким шагом медленно фланируют по перрону в направлении выхода. Чудаки, вы зачем толкались-то? Что вы хотели этим выиграть? Тут вообще ни секунды выигрыша Или вот сегодня. На 

## Итерация по страницам сайта

Каждая ссылка на страницу состоит из домена и даты: https://zadolba.li/20210928
* Доменая (неизменаяемая) часть: https://zadolba.li/
* Дата (изменяемая) часть: 20210928

Самая первая дата: 20090908

In [132]:
domain_part = 'https://zadolba.li/'

In [115]:
year1 = 2009
month1 = 9
day1 = 8
start_day = datetime.date(year1, month1, day1)

In [151]:
today = datetime.date.today()

In [125]:
timedelta = today - start_day
timedelta.days

4403

In [133]:
# Получим последние 10 ссылок
for days in range(timedelta.days)[:10]:
    
    current_day = today - datetime.timedelta(days=days)
    date_part = str(current_day).replace('-', '')
    print(domain_part + date_part)

https://zadolba.li/20210928
https://zadolba.li/20210927
https://zadolba.li/20210926
https://zadolba.li/20210925
https://zadolba.li/20210924
https://zadolba.li/20210923
https://zadolba.li/20210922
https://zadolba.li/20210921
https://zadolba.li/20210920
https://zadolba.li/20210919


## Собираем целый парсер

In [67]:
class ArticleParser:
    
    __domain_part = 'https://zadolba.li/'
    __first_date = '20090908'
    __time_out = 1
    
    def __init__(self, user_agent=None):
        self.__user_agent = user_agent
        self.table = pd.DataFrame(columns=['title','text', 'tags', 'date'])
    
    def start(self, parsing_type, clean_table=True):
        if clean_table:
            self.table = pd.DataFrame(columns=['title','text', 'tags', 'date'])
        
        start_day, last_day = self.__calculate_dates(parsing_type)
        days = (start_day - last_day).days
        day = start_day
        
        for current_days in tqdm.notebook.tnrange(1, days + 1):
            date_part = str(day).replace('-', '')
            link = ArticleParser.__domain_part + date_part
            html = self.__get_html(link)
            day = start_day - datetime.timedelta(days=current_days)
            if html is None:
                continue
            story_blocks = self.__get_story_blocks(html)
            for story_block in story_blocks:
                description = self.__get_description(story_block)
                self.table = self.table.append(description, ignore_index=True)
            time.sleep(ArticleParser.__time_out)

    def save_table(self, path):
        self.table.to_csv(path, sep=';', index=False)
    
        
    def __calculate_dates(self, parsing_type):
        if parsing_type[:4] == 'last':
            today = datetime.date.today()
            try:
                number = int(parsing_type.split(' ')[1])
            except ValueError:
                raise ValueError('The parsing type should be followed by a number (days)')
            last_day = today - datetime.timedelta(days=number)
            return today, last_day
        
        elif parsing_type[:5] == 'first':
            last_day = datetime.date(int(ArticleParser.__first_date[:4]), 
                                     int(ArticleParser.__first_date[4:6]), 
                                     int(ArticleParser.__first_date[6:8]))
            try:
                number = int(parsing_type.split(' ')[1])
            except ValueError:
                raise ValueError('The parsing type should be followed by a number (days)')
            start_day = last_day + datetime.timedelta(days=number)
            return start_day, last_day
        
        elif parsing_type == 'all':
            today = datetime.date.today()
            last_day = datetime.date(int(first_date[:4]), int(first_date[4:6]), int(first_date[6:8]))
            return today, last_day
            
        else:
            raise ValueError('Uncorrect pasring type. Parsing types starts with only these key-words: last or all')
    
    def __get_html(self, link):
        response = requests.get(link, headers={'User-Agent': self.__user_agent})
        if response.ok:
            return response.content
        else:
            return None
        
    def __get_story_blocks(self, html):
        soup = bs4.BeautifulSoup(html, 'html.parser')
        content_block = soup.find('div', {'class': 'content'})
        story_blocks = content_block.find_all('div', {'class': 'story'})
        return story_blocks
    
    def __get_description(self, story_block):

        try:
            title = story_block.h2.text.replace('\xa0', ' ')
        except AttributeError:
            title = None

        try:
            text = story_block.find('div', {'class': 'text'}).text.replace('\xa0', ' ').replace('\n', ' ').strip()
        except AttributeError:
            text = None

        try:
            html_tags = story_block.find('div', {'class': 'tags'}).ul.find_all('li')
            tags = [html_tag.a.text for html_tag in html_tags]
            tags = ', '.join(tags)
        except AttributeError:
            tags = None

        try:
            meta_block = story_block.find('div', {'class': 'meta'})
            datetime = meta_block.find('time').get('datetime')
            date = datetime.split('T')[0]
        except AttributeError:
            date = None

        return {'title': title, 'text': text, 'tags': tags, 'date': date}

In [68]:
parser = ArticleParser()

In [72]:
parser.start(parsing_type='first 50', clean_table=False)

  0%|          | 0/50 [00:00<?, ?it/s]

In [73]:
parser.table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   223 non-null    object
 1   text    223 non-null    object
 2   tags    201 non-null    object
 3   date    223 non-null    object
dtypes: object(4)
memory usage: 7.1+ KB


In [74]:
parser.table.sample(5)

,title,text,tags,date
105,Deep Blue,Четыре года назад я работал в отделе аудио/вид...,реклама,2009-10-20
57,Рацпредложение: напишите «не нажимать»,Работали мы с товарищем на складе по отгрузке ...,покупатели,2009-10-24
180,Зверский аппетит,"Работала как-то я на базе, которая обслуживала...","продавцы, общепит",2009-10-14
14,В поисках альтер-эго,Подрабатываю в ларьке типа «сад-огород». Кажды...,покупатели,2009-10-27
196,"И хрюкотали зелюки, как мюмзики в мове",Звонок в сервисный центр.— У меня телевизор та...,"телефонограммы, сервис",2009-10-12


In [75]:
parser.save_table(path='article_data.csv')